In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from keras.preprocessing import image

In [2]:
def train_datagen(dir):
    train = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        vertical_flip=True,
        rescale=1./255
    )

    train = train.flow_from_directory(
    dir,
    target_size=(150, 150),
    class_mode='categorical',
    batch_size=32
    )

    return train

In [3]:
def test_datagen(dir):

    test = ImageDataGenerator(
        rescale=1./255
    )

    test = test.flow_from_directory(
        dir,
        target_size=(150, 150),
        batch_size=32,
        class_mode="categorical"
    )

In [4]:
tr = train_datagen(r"E:\Dataset\wheat disease\data\train")

Found 13104 images belonging to 15 classes.


In [5]:
te = test_datagen(r"E:\Dataset\wheat disease\data\valid")

Found 300 images belonging to 15 classes.


In [6]:
def model():
    cnn = tf.keras.models.Sequential()

    #one
    cnn.add(tf.keras.layers.Conv2D(
    filters = 32,
    kernel_size = (3, 3),
    strides=(1, 1),
    activation="relu",
    input_shape = (150, 150, 3)
    ))

    cnn.add(tf.keras.layers.MaxPool2D(
    pool_size=(2, 2),
    strides=2
    ))


    #two
    cnn.add(tf.keras.layers.Conv2D(
    filters = 32,
    kernel_size = (3, 3),
    strides=(1, 1),
    activation="relu"
    ))

    cnn.add(tf.keras.layers.MaxPool2D(
    pool_size=(2, 2),
    strides=2
    ))


    #three
    cnn.add(tf.keras.layers.Conv2D(
    filters = 32,
    kernel_size = (3, 3),
    strides=(1, 1),
    activation="relu"
    ))

    cnn.add(tf.keras.layers.MaxPool2D(
    pool_size=(2, 2),
    strides=2
    ))

    cnn.add(tf.keras.layers.Flatten())

    cnn.add(tf.keras.layers.Dense(
        units = 512,
        activation="relu",
    ))

    cnn.add(tf.keras.layers.Dense(
        units=15,
        activation="softmax"
    ))

    cnn.compile(
        loss="categorical_crossentropy",
        optimizer = "adam",
        metrics=["accuracy"]
    )

    return cnn

In [7]:
cnn = model()

In [8]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 74, 74, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 36, 36, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 32)        9248      
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 17, 17, 32)        0

In [9]:
callback = tf.keras.callbacks.EarlyStopping(
    monitor='accuracy',
    patience=3)
cnn.fit(
    x=tr,
    batch_size=32,
    epochs=15,
    callbacks=[callback],
    validation_data=te
)

Epoch 1/15


410/410 [==============================] - 394s 958ms/step - loss: 2.0540 - accuracy: 0.3574
Epoch 2/15
410/410 [==============================] - 326s 796ms/step - loss: 1.5647 - accuracy: 0.5011
Epoch 3/15
410/410 [==============================] - 275s 672ms/step - loss: 1.3455 - accuracy: 0.5669
Epoch 4/15
410/410 [==============================] - 266s 649ms/step - loss: 1.1866 - accuracy: 0.6150
Epoch 5/15
410/410 [==============================] - 287s 701ms/step - loss: 1.0570 - accuracy: 0.6552
Epoch 6/15
410/410 [==============================] - 286s 698ms/step - loss: 0.9296 - accuracy: 0.6998
Epoch 7/15
410/410 [==============================] - 293s 716ms/step - loss: 0.8425 - accuracy: 0.7244
Epoch 8/15
410/410 [==============================] - 290s 708ms/step - loss: 0.7260 - accuracy: 0.7616
Epoch 9/15
410/410 [==============================] - 277s 674ms/step - loss: 0.6596 - accuracy: 0.7808
Epoch 10/15
410/410 [==============================] - 353s 86

In [10]:
cnn.save("second.h5")

C:\Users\tharu\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [34]:
test_image =image.load_img(r"E:\Dataset\wheat disease\data\test\septoria_test\septoria_14.png", target_size = (150, 150))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
print(test_image.shape)
result = cnn.predict(test_image)
print(result)

(1, 150, 150, 3)
1/1 [==============================] - 0s 29ms/step
[[0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
  0.000000e+00 1.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
  0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 1.904549e-15]]


In [33]:
ind = tr.class_indices
for i, j in ind.items():
    if result[0][j] == 1:
        print(result[0][j] , j)
        print(i)
    

1.0 14
Yellow Rust


In [26]:
ind

{'Aphid': 0,
 'Black Rust': 1,
 'Blast': 2,
 'Brown Rust': 3,
 'Common Root Rot': 4,
 'Fusarium Head Blight': 5,
 'Healthy': 6,
 'Leaf Blight': 7,
 'Mildew': 8,
 'Mite': 9,
 'Septoria': 10,
 'Smut': 11,
 'Stem fly': 12,
 'Tan spot': 13,
 'Yellow Rust': 14}